In [1]:
from transformers import pipeline
question_answer = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [2]:
context="Pink Floyd was an English rock band formed in London in 1965"
result = question_answer(question="When was Pink Floyd formed?", context=context)
result

{'score': 0.9961745142936707, 'start': 56, 'end': 60, 'answer': '1965'}

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

context_2= """Symptoms of COVID-19 are variable, 
            but often include fever, cough, fatigue, breathing difficulties, and loss of smell and taste. 
            Symptoms may begin one to fourteen days after exposure to the virus. 
            At least a third of people who are infected do not develop noticeable symptoms.
            [9] Of those people who develop noticeable symptoms enough to be classed as patients, 
            most (81%) develop mild to moderate symptoms (up to mild pneumonia), 
            while 14% develop severe symptoms (dyspnea, hypoxia, or more than 50% lung involvement on imaging), 
            and 5% suffer critical symptoms (respiratory failure, shock, or multiorgan dysfunction).
            [10] Older people are more likely to have severe symptoms. 
            Some people continue to experience a range of effects—known as long COVID—for months after recovery, 
            and damage to organs has been observed.
            [11] Multi-year studies are underway to further investigate the long-term effects of the disease."""

question_2="What are the symptoms of COVID-19 ?"
result_2 = question_answer(question=question_2, context=context_2,model="ktrapeznikov/biobert_v1.1_pubmed_squad_v2",tokenize="ktrapeznikov/biobert_v1.1_pubmed_squad_v2")
print(result_2)


{'score': 0.8357855081558228, 'start': 66, 'end': 140, 'answer': 'fever, cough, fatigue, breathing difficulties, and loss of smell and taste'}


In [15]:
import telebot

API_TOKEN = ''
bot = telebot.TeleBot(API_TOKEN)
@bot.message_handler(func=lambda message: True)
def reply_hi(message):
    bot.reply_to(message, "hi")
bot.polling()


good for token
good for bot
good for function
good for polling


In [7]:
import wikipedia
import telebot
API_TOKEN = ''

question_answer = pipeline('question-answering')

bot = telebot.TeleBot(API_TOKEN)
message_state=0
summary=""
subject=""


@bot.message_handler(func=lambda message: True)
def answering_bot(message):
    global message_state
    global summary
    global subject
    if message_state==0:
        bot.send_message(message.chat.id,"Hi, tell me a subject you wanted to talk about.")
        message_state=1
        return
    if message_state==1:
        try:
            wiki_rep=wikipedia.search(message.text)
            summary = wikipedia.summary(wiki_rep[0])
            bot.reply_to(message, "ask me a question about your subject")
            subject=message.text
            message_state=2
        except wikipedia.exceptions.PageError:
            bot.send_message(message.chat.id,"sorry I can't find any information about your subject. give me another subject if you want")
        except wikipedia.exceptions.DisambiguationError:
            bot.send_message(message.chat.id,"can you precise your need?")
        return
    if message_state==2:
        result = question_answer(question=message.text, context=summary)
        bot.reply_to(message, list(result.values())[3])
        bot.send_message(message.chat.id,"Do you want to ask me another question about this subject?")
        message_state=3
        return
    if message_state==3:
        if message.text in ["?"]:
            message_state=1
        if message.text in ["Yes","yes","YES","y","Y"]:
            bot.send_message(message.chat.id, "ask me a question about "+subject)
            message_state=2
        elif message.text in ["No","NO","n","N"]:
            bot.send_message(message.chat.id, "tell me a subject you wanted to talk about.")  
            message_state=0
        else:
            bot.reply_to(message, "I did not understand your response, can you response with yes or no?")
            message_state=3
        return

bot.polling()

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)
